In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta, date

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals as F
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import StaticAssets
from quantopian.research import run_pipeline


DEBUG = True


BILLION = 1000000000

MARKER_INDEX = 0
COLOR_INDEX = 1
DASH_INDEX = 2

INCOME = 1
BALANCE = 2
CF = 3
MISC = 4
RATIO = 5
GROWTH = 6
YIELD = 7

CLOSE_COL = 'close'

# colors: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
ATTR = {
    'close': ('.', 'k', tuple()),
    'market_cap': ('.', 'k', tuple()),
    'shares': ('.', 'deeppink', tuple()),
    
    'revenue': ('s', 'orange', (2,2)),
    'gross_profit' : ('p', 'sandybrown', (2,2)),
    'operating_income' : ('v', 'lightblue', (2,2)),
    'net_income': ('o', 'b', (2,2)),
    
    'ps': ('s', 'orange', (2,2)),
    'pe': ('o', 'b', (2,2)),
    'pfcf': ('^', 'g', (2,2)),
    'pb': ('*', 'grey', (2,2)),
    'ev/sales': ('p', 'sandybrown', (2,2)),
    
    'equity': ('*', 'grey', (2,2)),
    'cash' : ('+', 'g', (2,2)),
    'short_term_liabilities': ('>', 'red', (2,2)),
    'long_term_liabilities': ('<', 'darkred', (2,2)),
    
    'fcf': ('^', 'g', (2,2)),
    
    'revenue_growth':('s', 'orange', (2,2)),
    'gross_margin': ('p', 'sandybrown', (2,2)),
    'eps_growth': ('v', 'lightblue', (2,2)),
    'net_margin': ('o', 'b', (2,2)),
    'rnd/sales': ('^', 'tan', (2,2)),

    'sales_yield':('s', 'orange', (2,2)),
    'cfo_yield': ('p', 'sandybrown', (2,2)),
    'earning_yield': ('v', 'lightblue', (2,2)),
    'fcf_yield': ('o', 'b', (2,2)),
    'book_yield': ('^', 'tan', (2,2)),
}


def make_pipeline(tickers):
    mask = StaticAssets(tickers)
    
    close = USEquityPricing.close.latest
    market_cap = F.market_cap.latest
    
    # ratios
    ps = F.ps_ratio.latest
    pe = F.pe_ratio.latest
    pfcf = F.fcf_ratio.latest
    pb = F.pb_ratio.latest
    
    ev = F.enterprise_value.latest
    
    # income statement
    revenue = F.total_revenue.latest
    gross_profit = F.gross_profit.latest
    operating_income = F.operating_income.latest
    net_income = F.net_income_common_stockholders.latest
    
    rnd = F.research_and_development.latest
    operating_expense = F.operating_expense.latest
    
    # balance sheet statement
    equity = F.stockholders_equity.latest
    cash = F.cash_and_cash_equivalents.latest
    stl = F.current_liabilities.latest
    ltl = F.long_term_debt.latest
    
    # growth
    revenue_growth = F.revenue_growth.latest
    gross_margin = F.gross_margin.latest
    eps_growth = F.diluted_eps_growth.latest
    net_margin = F.net_margin.latest
    fcf_return = F.cash_return.latest
    
    # yield
    book_yield = F.book_value_yield.latest
    cfo_yield = F.cf_yield.latest
    earning_yield = F.earning_yield.latest
    fcf_yield = F.fcf_yield.latest
    sales_yield = F.sales_yield.latest
    
    
    ##equity = assets - stl - ltl
    
    # cash flow statement
    fcf = F.free_cash_flow.latest
    
    shares = F.shares_outstanding.latest
    
    period_ending_date = F.period_ending_date.latest
    
    pipe = Pipeline(
        columns={
            'close': close,
            'period_ending_date': period_ending_date,
            'market_cap': market_cap,
            'shares': shares,

            'revenue': revenue,
            'gross_profit' : gross_profit,
            'operating_income': operating_income,
            'net_income': net_income,
            
            'ps': ps,
            'pe': pe,
            'pfcf': pfcf,
            'pb': pb,
            
            'ev/sales': ev / revenue,
            
            'equity': equity,
            'cash' : cash,
            'short_term_liabilities': stl,
            'long_term_liabilities': ltl,
            
            'fcf': fcf,
            
            'revenue_growth': revenue_growth,
            'gross_margin': gross_margin, 
            'eps_growth': eps_growth,
            'net_margin': net_margin,
            'rnd/sales': rnd / revenue,
            #'fcf_return': fcf_return,
            
            'book_yield': book_yield,
            'cfo_yield': cfo_yield,
            'earning_yield': earning_yield,
            'fcf_yield': fcf_yield,
            'sales_yield': sales_yield,
            
        },
        screen=mask
    )
    return pipe


def prune_pipeline(pipeline, ticker):
    
    period_ending_dates = pd.to_datetime(pd.unique(pipeline['period_ending_date']))
    other_dates = period_ending_dates + timedelta(days=3)
    market_dates = period_ending_dates.union(other_dates)

    is_market_date = [True] * len(pipeline.index) #pipeline.index.get_level_values(0).isin(market_dates)
    is_ticker = pipeline.index.get_level_values(1) == symbols(ticker).sid
    return pipeline[is_market_date & is_ticker].reset_index()


def graph_stacked(pipeline, ax, columns, state):
    # TODO: generalize by passing in columns to graph and normalize boolean
    #  and randomly picking color/marker/shape
    
    def normalize_pipeline(col):
        denom = BILLION
        return pipeline[col] / denom
            
    x = [x.date() for x in pipeline['level_0']]
    y = [normalize_pipeline(column) for column in columns]
    colors = [ATTR[column][COLOR_INDEX] for column in columns]
    
    ax.stackplot(x, *y, baseline='zero', colors=colors)
    ax.legend(columns, loc='upper left')   
    

def graph(pipeline, ax, columns, state):
    
    def normalize_pipeline(col):
        if state in (RATIO, MISC, GROWTH):
            return pipeline[col]
        denom = BILLION
        return pipeline[col] / denom
         
    x = [x.date() for x in pipeline['level_0']]
    
    for column in columns:
        y = normalize_pipeline(column)
        ax.plot(x, y, 
                marker=ATTR[column][MARKER_INDEX],
                color=ATTR[column][COLOR_INDEX],
                dashes=ATTR[column][DASH_INDEX])
    ax.legend(columns, loc='upper left')


def get_columns(state):
    columns = []
    
    if state == MISC:
        columns += [
            #'close',
            
            'shares',
            #'spy',  # TODO
            
        ]

    if state == INCOME:
        columns += [
            'revenue',
            'gross_profit',
            'operating_income',
            'net_income',
            #'fcf',
        ]
        
    if state == RATIO:
        columns += [
            'ps',
            #'ev/sales',
            #'pe',
            #'pfcf',
            #'pb',
        ]

    if state == BALANCE:
        columns += [
            #'cash',
            'short_term_liabilities',
            'long_term_liabilities',
            'equity',
            #'market_cap',
        ]

    if state == CF:
        columns += [
            'fcf', 
        ]
        
    if state == GROWTH:
        columns += [
            'revenue_growth',
            'gross_margin',
            #'eps_growth',
            'net_margin',
            #'fcf_return',
            #'rnd/sales'
        ]

    if state == YIELD:
        columns += [            
            #'book_yield',
            'cfo_yield',
            'earning_yield',
            'fcf_yield',
            'sales_yield',
        ]
        
    return columns
    

OVERLAY_CLOSE = True

def main(tickers, start_date='2020-08-01', states=(INCOME, RATIO, BALANCE, MISC)):

    FIG_WIDTH = 15
    FIG_HEIGHT = len(tickers)*45

    yest = (datetime.now() - timedelta(days=1)).date()

    _pipeline = run_pipeline(make_pipeline(tickers), start_date, yest)

    fig, axs = plt.subplots(len(tickers)*(len(states)), 1, figsize=(FIG_WIDTH,FIG_HEIGHT))
    plt.subplots_adjust(hspace=.5)

    for i, ticker in enumerate(tickers):
        pipeline = prune_pipeline(_pipeline, ticker)
        
        # other plots
        for j, state in enumerate(states):
            index = (len(states))*i + (j)
            axs[index].set_title(ticker)
            try:
                if state == BALANCE:
                    graph_stacked(pipeline, axs[index], get_columns(state), state)
                else:
                    graph(pipeline, axs[index], get_columns(state), state)
            except Exception as e:
                print(ticker)        
                print('ERROR: ', e)
                print(pipeline.tail(1))
                if DEBUG:
                    raise e
                continue

            # overlay stock price plot
            if OVERLAY_CLOSE:
                y = pipeline[CLOSE_COL]
                x = [x.date() for x in pipeline['level_0']]
                ax_twin = axs[index].twinx()
                ax_twin.plot(x, y, marker='.', color='k', linewidth=1)

            
tickers = symbols([
    #'x',
    
    #'aapl',
    #'msft',
    #'goog',
    'amzn',
    #'nflx',
    #'znga',
    #'brkb',
    #'dis',
    #'t',
    #'axp',
    #'v',
    #'ma',
    
    #'sq',
    #'pypl',
    #'apps',
    #'shop',
    #'spot',
    #'jpm',
    'mdb',
    'snap',
    'etsy',
    #'pd',
    #'ayx',
    'ddog',
    'okta',
    'fsly',
    'net',
    'zs',
    'docu',
    'crwd',
    'lmnd',
    #'zm',
    #'twlo',
    
    #'txn',
    #'intc',
    
    #'eb',
    #'snap',
    #'docu',
    #'tdoc'
    #'m',
    #'bset',
    #'gme',
    
    ## NO DATA
    #'spot',
    #'plnhf',
    #'tcnnf'
    
    #'spy',
    
    '''
    'mdb',
    'sq',
    'pypl',
    'shop',
    'eb',
    'snap',
    'docu',
    'tdoc'
    'upwk',
    'se',
    'etsy',
    'meli',
    'nvta',
    'ayx',
    'fsly',
    'crwd',
    'fvrr',
    'api',
    'roku',
    'isrg',
    #'spce',  # outlier
    'spot',
    'net',
    'chgg',
    'twlo',
    'ddog',
    'shop',
    'work',
    'wday',
    'splk',
    'tsla',
    'nflx',
    'yext',
    'fb',
    'uber',
    'lyft',
    'twtr',
    'okta',
    'apps',
    'z',
    'rdfn',
    'cldr',
    'crm',
    'pd',
    'zm',
    'dbx',
    'net',
    'bl',
    'zen',
    'znga',
    'pton',
    'snow',
    '''
])

start_date='2016-08-01'
states = (INCOME, GROWTH, YIELD, RATIO, BALANCE, MISC)

main(tickers, start_date=start_date, states=states )
